## SQL helper queries

Analyzing the number of active connections to the database

In [ ]:
WITH
    cfg AS (
        SELECT
            current_setting('max_connections')::int                 AS max_conn,
            current_setting('superuser_reserved_connections')::int  AS super_res
    ),
    used AS (
        SELECT count(*) AS active_cnt FROM pg_stat_activity
    )
SELECT
    super_res,
    active_cnt,
    (max_conn - super_res - active_cnt) AS free_for_normal_users
FROM cfg, used;


Closes all user connections to the database.

In [ ]:
SELECT pg_terminate_backend(pid)
FROM   pg_stat_activity
WHERE  usename        = current_user      -- only terminate connections of your user
  AND  pid <> pg_backend_pid();           -- do not terminate the current session


Get information about all active connections

In [ ]:
SELECT  pid,               -- process id (useful for pg_terminate_backend)
        usename,           -- owning role / user
        datname,           -- database name
        application_name,  -- e.g. psql, psycopg2, JDBC
        client_addr,       -- client IP (NULL = local)
        backend_start,     -- when connection was established
        state,             -- active | idle | idle in txn | ...
        query              -- last statement (NULL if no privilege)
FROM    pg_stat_activity
ORDER BY usename, pid;
